In [1]:
import os
import glob
import numpy as np

from pinf.analytical_field import get_analytic_b_field
from pinf.unpack import load_cube
from pinf.plot import pv_plot
from pinf.performance_metrics import metrics

In [2]:
b = get_analytic_b_field(n=1, m=1, l=0.3, psi=np.pi/2, resolution=64, bounds=[-1, 1, -1, 1, 0, 2])

In [3]:
pv_plot(b, vtk_path=f'./output/vtk/b.vtk', points=((16, 49, 8), (16, 49, 8)), title='LL', overwrite=True, imgpath=f'./output/plot/b.pdf')
metrics(b, b, base_path=f'./output/eval/b.txt')

Potential Field: 100%|██████████| 6/6 [00:01<00:00,  3.12it/s]


In [4]:
for f in sorted(glob.glob('run/dim256_bin1_pfTrue_ld0.1_lf0.1/fields_*.nf2')):
    title = 'PINN' + '_' + os.path.basename(f).split('.')[0][7:]
    B = load_cube(f)
    pv_plot(B, vtk_path=f'./output/vtk/{title}.vtk', points=((16, 49, 8), (16, 49, 8)), title=title, overwrite=True, imgpath=f'./output/plot/{title}.pdf')
    metrics(B, b, base_path=f'./output/eval/{title}.txt')

40 24


Potential Field: 100%|██████████| 6/6 [00:01<00:00,  3.17it/s]


RuntimeError: DataLoader worker (pid 39889) is killed by signal: Segmentation fault: 11. 